yt has capabilities for writing 2D and 3D uniformly gridded data generated from datasets to FITS files. This is via the `FITSImageData` class. We'll test these capabilities out on an Athena dataset.

In [ ]:
import yt
from yt.utilities.fits_image import FITSImageData, FITSProjection

In [ ]:
ds = yt.load("MHDSloshing/virgo_low_res.0054.vtk", parameters={"length_unit":(1.0,"Mpc"),
                                                               "mass_unit":(1.0e14,"Msun"),
                                                               "time_unit":(1.0,"Myr")})

## Creating FITS images from Slices and Projections

There are several ways to make a `FITSImageData` instance. The most intuitive ways are to use the `FITSSlice`, `FITSProjection`, `FITSOffAxisSlice`, and `FITSOffAxisProjection` classes to write slices and projections directly to FITS. To demonstrate a useful example of creating a FITS file, let's first make a `ProjectionPlot`:

In [ ]:
prj = yt.ProjectionPlot(ds, "z", ["temperature"], weight_field="density", width=(500.,"kpc"))
prj.show()

Suppose that we wanted to write this projection to a FITS file for analysis and visualization in other programs, such as ds9. We can do that using `FITSProjection`:

In [ ]:
prj_fits = FITSProjection(ds, "z", ["temperature"], weight_field="density")

which took the same parameters as `ProjectionPlot` except the width, because `FITSProjection` and `FITSSlice` always make slices and projections of the width of the domain size, at the finest resolution available in the simulation, in a unit determined to be appropriate for the physical size of the dataset.

We can call a number of the [AstroPy `HDUList`](http://astropy.readthedocs.org/en/latest/io/fits/api/hdulists.html) class's methods from a `FITSImageData` object. For example, `info` shows us the contents of the virtual FITS file:

In [ ]:
prj_fits.info()

We can also look at the header for a particular field:

In [ ]:
prj_fits["temperature"].header

where we can see that the temperature units are in Kelvin and the cell widths are in kiloparsecs. If we want the raw image data with units, we can call `get_data`:

In [ ]:
prj_fits.get_data("temperature")

We can use the `set_unit` method to change the units of a particular field:

In [ ]:
prj_fits.set_unit("temperature","R")
prj_fits.get_data("temperature")

The image can be written to disk using the `writeto` method:

In [ ]:
prj_fits.writeto("sloshing.fits", clobber=True)

Since yt can read FITS image files, it can be loaded up just like any other dataset:

In [ ]:
ds2 = yt.load("sloshing.fits")

and we can make a `SlicePlot` of the 2D image, which shows the same data as the previous image:

In [ ]:
slc2 = yt.SlicePlot(ds2, "z", ["temperature"], width=(500.,"kpc"))
slc2.set_log("temperature", True)
slc2.show()

## Using `FITSImageData` directly

If you want more fine-grained control over what goes into the FITS file, you can call `FITSImageData` directly, with various kinds of inputs. For example, you could use a `FixedResolutionBuffer`, and specify you want the units in parsecs instead:

In [ ]:
slc3 = ds.slice(0, 0.0)
frb = slc3.to_frb((500.,"kpc"), 800)
fid_frb = FITSImageData(frb, fields=["density","temperature"], units="pc")

A 3D FITS cube can also be created from a covering grid:

In [ ]:
cvg = ds.covering_grid(ds.index.max_level, [-0.5,-0.5,-0.5], [64, 64, 64], fields=["density","temperature"])
fid_cvg = FITSImageData(cvg, fields=["density","temperature"], units="Mpc")

## Other `FITSImageData` Methods

A `FITSImageData` instance can be generated from one previously written to disk using the `from_file` classmethod:

In [ ]:
fid = FITSImageData.from_file("sloshing.fits")
fid.info()

Multiple `FITSImageData` can be combined to create a new one, provided that the coordinate information is the same:

In [ ]:
prj_fits2 = FITSProjection(ds, "z", ["density"])
prj_fits3 = FITSImageData.from_images([prj_fits, prj_fits2])
prj_fits3.info()

Alternatively, individual fields can be popped as well:

In [ ]:
dens_fits = prj_fits3.pop("density")

In [ ]:
dens_fits.info()

In [ ]:
prj_fits3.info()

So far, the FITS images we have shown have linear spatial coordinates. One may want to take a projection of an object and make a crude mock observation out of it, with celestial coordinates. For this, we can use the `create_sky_wcs` method. Specify a center (RA, Dec) coordinate in degrees, as well as a linear scale in terms of angle per distance:

In [ ]:
sky_center = [30.,45.] # in degrees
sky_scale = (2.5, "arcsec/kpc") # could also use a YTQuantity
prj_fits.create_sky_wcs(sky_center, sky_scale, ctype=["RA---TAN","DEC--TAN"])

By the default, a tangent RA/Dec projection is used, but one could also use another projection using the `ctype` keyword. We can now look at the header and see it has the appropriate WCS:

In [ ]:
prj_fits["temperature"].header

Finally, we can add header keywords to a single field or for all fields in the FITS image using `update_header`:

In [ ]:
fid_frb.update_header("all", "time", 0.1) # Update all the fields
fid_frb.update_header("temperature", "scale", "Rankine") # Update just one field

In [ ]:
print (fid_frb["density"].header["time"])
print (fid_frb["temperature"].header["scale"])